In [1]:
import pandas as pd

In [2]:
#Leitura dos dados para cálculo dos valores de venda e compra
df1 = pd.read_csv('Teste casa dos ventos/contracts.csv', sep=';')

In [3]:
df1['volume_mwh'] = df1['volume_mwm']*(31*24)
df1['valor'] = df1.price*(df1.volume_mwh)
df1.rename(columns = {'party' : 'park'}, inplace = True)
pd.set_option('display.max_rows', 144)

In [4]:
#Filtrando e separando em dados de compra e venda
df_filtro = df1.groupby(['park', 'month', 'type']).agg({'volume_mwh':'sum'}).reset_index()
df_filtro2 = df1.groupby(['park', 'month', 'type']).agg({'valor':'sum'}).reset_index()
df_compra1 = df_filtro[((df_filtro['type'] == 'compra')&(df_filtro['month'] < 7))]
df_compra2 = df_filtro2[((df_filtro2['type'] == 'compra')&(df_filtro['month'] < 7))]
df_venda1 = df_filtro[((df_filtro['type'] == 'venda')&(df_filtro['month'] < 7))]
df_venda2 = df_filtro2[((df_filtro2['type'] == 'venda')&(df_filtro['month'] < 7))]
df_compra = df_compra1.merge(df_compra2, how = 'left', on = ['park', 'month'])
df_venda = df_venda1.merge(df_venda2, how = 'left', on = ['park', 'month'])
del df_compra['type_y']
del df_venda['type_y']
df_compra.rename(columns = {'type_x' : 'type'}, inplace = True)
df_venda.rename(columns = {'type_x' : 'type'}, inplace = True)

In [5]:
#Leitura dos dados para cálculo da liquidação
df_geracao = pd.read_csv('Teste casa dos ventos/geracao_realizada.csv')
df_pld = pd.read_csv('Teste casa dos ventos/pld.csv')

In [6]:
#Transformando a string em data nos dados da geração horária
df_geracao['date'] = pd.to_datetime(df_geracao['datetime'])
df_geracao['month'] = df_geracao.date.dt.month
del df_geracao['datetime']
df_geracao['park'] = df_geracao['park'].str.lower()

In [7]:
#Transformando a string em data nos dados de PLD
df_pld['date'] = pd.to_datetime(df_pld['datetime'])
df_pld['month'] = df_pld.date.dt.month
del df_pld['datetime']
df_pld = df_pld.sort_values(by=['date', 'month'], ignore_index= True)
df_pld = df_pld[(df_pld['submercado'] == 'NORDESTE')]

In [8]:
#Unindo os dados de geração horária aos dados de PLD horário para posterior cálculo da liquidação
df_liquidacao = df_geracao.merge(df_pld, how = 'left', on = ['date', 'month'])
del df_liquidacao['submercado']

In [9]:
#Cálculo das grandezas necessárias
df_compra['volume_mwh_compra_horario'] = df_compra['volume_mwh']/(31*24)
df_venda['volume_mwh_venda_horario'] = df_venda['volume_mwh']/(31*24)
df_liquidacao = df_liquidacao.merge(df_compra, how = 'left', on = ['park', 'month'])
df_liquidacao = df_liquidacao.merge(df_venda, how = 'left', on = ['park', 'month'], suffixes=('_compra', '_venda'))
del df_liquidacao['type_compra']
del df_liquidacao['type_venda']
del df_liquidacao['volume_mwh_compra']
del df_liquidacao['valor_compra']
del df_liquidacao['volume_mwh_venda']
del df_liquidacao['valor_venda']
df_liquidacao['volume_mwh_compra_horario'] = df_liquidacao['volume_mwh_compra_horario'].fillna(0)
df_liquidacao['receita_exposicao'] = df_liquidacao['generation_mwh']*df_liquidacao['pld']
df_liquidacao['balanco_energetico'] = df_liquidacao['generation_mwh'] + df_liquidacao['volume_mwh_compra_horario'] - df_liquidacao['volume_mwh_venda_horario']
df_liquidacao['liquidacao'] = (df_liquidacao['generation_mwh'] + df_liquidacao['volume_mwh_compra_horario'] - df_liquidacao['volume_mwh_venda_horario'])*df_liquidacao['pld']

In [10]:
#Separando em diferentes dataframes para cada dado
df_exposicao = df_liquidacao.groupby(['park', 'month']).agg({'receita_exposicao':'sum'}).reset_index()
df_geracao = df_liquidacao.groupby(['park', 'month']).agg({'generation_mwh':'sum'}).reset_index()
df_balanco = df_liquidacao.groupby(['park', 'month']).agg({'balanco_energetico':'sum'}).reset_index() 
df_liquidacao = df_liquidacao.groupby(['park', 'month']).agg({'liquidacao':'sum'}).reset_index()

In [11]:
#Unindo todos os dados separados por mês e parque
df_geral = df_geracao.merge(df_compra, how = 'left', on = ['park', 'month'])
df_geral = df_geral.merge(df_venda, how = 'left', on = ['park', 'month'], suffixes=('_compra', '_venda'))
df_geral = df_geral.merge(df_exposicao, how = 'left', on = ['park', 'month'])
df_geral = df_geral.merge(df_balanco, how = 'left', on = ['park', 'month'])
df_geral = df_geral.merge(df_liquidacao, how = 'left', on = ['park', 'month'])
del df_geral['type_compra']
del df_geral['type_venda']
del df_geral['volume_mwh_compra_horario']
del df_geral['volume_mwh_venda_horario']
df_geral['volume_mwh_compra'] = df_geral['volume_mwh_compra'].fillna(0)
df_geral['valor_compra'] = df_geral['valor_compra'].fillna(0)

In [12]:
#Cálculo das receitas
Receita_contratos = df_geral['valor_venda'] - df_geral['valor_compra']
Receita = df_geral['valor_venda'] - df_geral['valor_compra'] + df_geral['liquidacao']
df_geral['receita_contratos'] = Receita_contratos
df_geral['receita'] = Receita

In [13]:
#Arquivos de saída
df_geral.to_csv('parte1.csv')
df_geral.to_excel('parte1.xlsx')